## Выполнение 1 задания, 1 подпункт

In [1]:
!pip3 install pandas

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)


You should consider upgrading via the 'T:\code_t\Blanks\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import os
import pandas as pd
import re

text_dir = 'data/path/to/text'
annotation_dir = 'data/path/to/annotation'
tags_dir = 'data/path/to/tags'

# список файлов в каждой папке
text_files = sorted(os.listdir(text_dir))
annotation_files = sorted(os.listdir(annotation_dir))
tags_files = sorted(os.listdir(tags_dir))

# Проверка на совпадение кол-ва файлов.
assert len(text_files) == len(annotation_files) == len(tags_files), "Количество файлов в папках не совпадает"

data = pd.DataFrame({
    'text_file': text_files,
    'annotation_file': annotation_files,
    'tags_file': tags_files
})

data.head()

## Проверка на "Заспамленность" 1.2

In [ ]:
spam_words = ['купить', 'бесплатно', 'срочно', 'акция', 'выиграй']

In [ ]:
def calculate_spam_score(text):
    text = text.lower()
    count = sum(text.count(word) for word in spam_words)
    return count

# Функция для подсчёта символов с и без пробелов
def count_chars(text):
    with_spaces = len(text)
    without_spaces = len(text.replace(" ", ""))
    return with_spaces, without_spaces

# Функция для подсчёта ключевых слов
def count_keywords(tags):
    keywords = tags.split(',')  # Предполагаю, что ключевые слова разделены запятыми
    return len([kw.strip() for kw in keywords if kw.strip()])

In [ ]:
# Списки для хранения значений
spam_scores = []
annotation_lengths = []
keywords_counts = []
chars_with_spaces = []
chars_without_spaces = []

In [ ]:
for idx, row in data.iterrows():
    with open(os.path.join(text_dir, row['text_file']), 'r', encoding='utf-8') as f:
        text = f.read()
    
    with open(os.path.join(annotation_dir, row['annotation_file']), 'r', encoding='utf-8') as f:
        annotation = f.read()
    
    with open(os.path.join(tags_dir, row['tags_file']), 'r', encoding='utf-8') as f:
        tags = f.read()
        
    spam_scores.append(calculate_spam_score(text))
    annotation_lengths.append(len(annotation))
    keywords_counts.append(count_keywords(tags))
    w_spaces, wo_spaces = count_chars(text)
    chars_with_spaces.append(w_spaces)
    chars_without_spaces.append(wo_spaces)

In [ ]:
# Создание датафрейма под вычисленные параметры из функций

data['spam_score'] = spam_scores
data['annotation_length'] = annotation_lengths
data['keywords_count'] = keywords_counts
data['chars_with_spaces'] = chars_with_spaces
data['chars_without_spaces'] = chars_without_spaces

data.head()

## Статистический анализ 1.3

In [3]:
!pip3 install matplotlib seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


You should consider upgrading via the 'T:\code_t\Blanks\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style="whitegrid")

def plot_distribution(column, title):
    plt.figure(figsize=(10,6))
    sns.histplot(data[column], bins=30, kde=True)
    plt.title(title)
    plt.show()

plot_distribution('spam_score', 'Распределение заспамленности')
plot_distribution('annotation_length', 'Распределение длины аннотации')
plot_distribution('keywords_count', 'Распределение количества ключевых слов')
plot_distribution('chars_with_spaces', 'Распределение количества символов с пробелами')
plot_distribution('chars_without_spaces', 'Распределение количества символов без пробелов')

Далее надо проводить очистку датасета и сохранение его, но у меня нету данных, поэтому просто вставил заготовку, которая была. (модифицировал ее под задачу с помощью ChatGPT)

In [ ]:
# Рассчитаем медиану и 75-й перцентиль для каждого параметра
thresholds = {
    'spam_score': data['spam_score'].median(),
    'annotation_length': data['annotation_length'].median(),
    'keywords_count': data['keywords_count'].median(),
    'chars_with_spaces': data['chars_with_spaces'].median(),
    'chars_without_spaces': data['chars_without_spaces'].median()
}

print("Пороговые значения (медиана):")
for key, value in thresholds.items():
    print(f"{key}: {value}")


In [ ]:
# Рассчитаем 75-й перцентиль
for key in thresholds.keys():
    thresholds[key] = data[key].quantile(0.75)

print("\nПороговые значения (75-й перцентиль):")
for key, value in thresholds.items():
    print(f"{key}: {value}")


In [ ]:
# Функция для фильтрации
def filter_data(df, thresholds):
    filtered_df = df[
        (df['spam_score'] <= thresholds['spam_score']) &
        (df['annotation_length'] >= thresholds['annotation_length']) &
        (df['keywords_count'] >= thresholds['keywords_count']) &
        (df['chars_with_spaces'] >= thresholds['chars_with_spaces']) &
        (df['chars_without_spaces'] >= thresholds['chars_without_spaces'])
    ]
    return filtered_df

# Применяем фильтрацию
filtered_data = filter_data(data, thresholds)

print(f"Количество статей до фильтрации: {len(data)}")
print(f"Количество статей после фильтрации: {len(filtered_data)}")

In [ ]:
# Создадим новую папку для отфильтрованных данных
filtered_text_dir = 'path/to/filtered_text'
filtered_annotation_dir = 'path/to/filtered_annotation'
filtered_tags_dir = 'path/to/filtered_tags'

os.makedirs(filtered_text_dir, exist_ok=True)
os.makedirs(filtered_annotation_dir, exist_ok=True)
os.makedirs(filtered_tags_dir, exist_ok=True)

# Копируем файлы
import shutil

for idx, row in filtered_data.iterrows():
    shutil.copy(os.path.join(text_dir, row['text_file']), os.path.join(filtered_text_dir, row['text_file']))
    shutil.copy(os.path.join(annotation_dir, row['annotation_file']), os.path.join(filtered_annotation_dir, row['annotation_file']))
    shutil.copy(os.path.join(tags_dir, row['tags_file']), os.path.join(filtered_tags_dir, row['tags_file']))

In [ ]:
filtered_data.to_csv('filtered_dataset.csv', index=False)